In [1]:
import glob
import os
import sys

orig_dir = os.getcwd()

os.chdir("../")
sys.path.append(os.getcwd())
import speech_commands_v2_ds as sc

os.chdir(orig_dir)

2024-04-22 15:20:21.037772: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 15:20:21.063031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 15:20:21.541507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
datapath = "../data/speech_commands_v2"

# Download the UrbanSound8K dataset
if not os.path.exists(datapath):
    os.makedirs(datapath)
    if not os.path.exists(datapath + "_raw"):
        os.makedirs(datapath + "_raw")
        # !wget "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz"
        !tar -xvf speech_commands_v0.02.tar.gz -C {datapath}_raw

    cnt = 0
    sum = len(glob.glob(f"{datapath}_raw/*/*"))
    for file in glob.glob(f"{datapath}_raw/*/*"):
        os.makedirs("/".join(file.split("/")[:-1]).replace(datapath + "_raw", datapath), exist_ok=True)
        fixed_file = f"{datapath}/{file.split('/')[-2]}/{file.split('/')[-1]}"
        os.system(f"ffmpeg -i {file} -acodec pcm_s16le -ar 16000 {fixed_file} > /dev/null 2>&1")
        cnt += 1
        if cnt % 100 == 0:
            print(f"Processed {cnt/sum} files")
    !rm -rf {datapath}_raw
    !rm -rf speech_commands_v0.02.tar.gz

In [ ]:
ds_dict = get_datasets(datapath)
train_ds = ds_dict["train"]
valid_ds = ds_dict["valid"]
test_ds = ds_dict["test"]

In [ ]:
# build the most simple  neural network to train on this data
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Input(shape=(16000,)),
    layers.Reshape((16000, 1)),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(36, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

model.fit(train_ds.batch(128), epochs=10)

In [ ]:
N = 41
D = 32
H = 4
layers = 2

layers*3*(3*N*D*H + (N**2)*D*H + 4*N*D**2)

In [ ]:
# print the size of all files in the datapath
import os
import numpy as np

def get_size_of_files(datapath):
    sizes = []
    for root, dirs, files in os.walk(datapath):
        for file in files:
            size = os.path.getsize(os.path.join(root, file))
            sizes.append(size)
            if size == 0:
                print(os.path.join(root, file))
    return np.array(sizes)

sizes = get_size_of_files(datapath)